In [1]:
#!python -m pip install cohere --upgrade
#!pip install -q -U google-generativeai
#!pip install anthropic

In [2]:
import os 
import ollama 
import cohere 
import anthropic 
from openai import OpenAI
import google.generativeai as genai
from dotenv import load_dotenv 
from IPython.display import Markdown, display, update_display

In [3]:
load_dotenv()

True

In [4]:
openai = OpenAI()  
co_api_key = os.getenv('COHERE_API_KEY') 
client = anthropic.Anthropic() 
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

In [5]:
system_prompt = "You are a helpful technical tutor who answers questions about python code, software engineering, data science and LLMs"

In [6]:
user_prompt = "Please give a detailed explanation to the following question: "

In [7]:
my_question = "Please enter your question: Please explain what this code does and why: yield from {book.get('author') for book in books if book.get('author')}"

In [8]:
def get_openai_response(question): 
    messages= [{"role": "system", "content": system_prompt}, {"role": "user", "content": user_prompt + question}]
    
    stream = openai.chat.completions.create(model='gpt-4o-mini', messages=messages,stream=True)
    
    response = "# OpenAI Response \n\n"
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [9]:
def get_ollama_response(question): 
    messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": user_prompt + question}] 

    response = ""
    response = ollama.chat(model="llama3.2", messages=messages)
    reply = f"# Ollama Response \n\n {response['message']['content']} "
    display(Markdown(reply))

In [10]:
def get_anthropic_response(question): 
    client = anthropic.Anthropic() 
    
    response = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=1000,
    temperature=0,
    system= system_prompt,   
    messages=[{"role": "user", "content": [{"type": "text","text": question}]}]
    )
 
    reply = f"# Anthropic Response \n\n{response.content[0].text}"
    display(Markdown(reply))

In [11]:
def get_cohere_response(question): 
    co = cohere.ClientV2(co_api_key)
    messages= [{"role": "system", "content": system_prompt}, {"role": "user", "content": user_prompt + question}] 

    response = co.chat(
        model="command-r-plus", 
        messages=messages
    )
    #print(response.message.content[0].text) 
    reply = f"# Cohere Response\n\n {response.message.content[0].text}" 
    display(Markdown(reply))

In [12]:
def get_gemini_response(question): 
    genai.configure(api_key=GEMINI_API_KEY)
    model = genai.GenerativeModel("gemini-1.5-flash")
    response = model.generate_content(my_question)
    #print(response.text)
    resp = f"# Gemini Response \n\n{response.text}"
    display(Markdown(resp))

In [13]:
get_openai_response(my_question)
get_ollama_response(my_question)
get_anthropic_response(my_question)
get_cohere_response(my_question)
get_gemini_response(my_question)

# OpenAI Response 

Certainly! Let's break down the provided line of code step by step to understand what it does, and then we'll discuss why it might be implemented in this way.

### Code Analysis

The code you're looking at is a Python generator expression using the `yield from` statement:
python
yield from {book.get('author') for book in books if book.get('author')}


#### Components of the Code

1. **`{ ... }`**: 
   - This denotes a set comprehension in Python. Set comprehensions are similar to list comprehensions but create a set (an unordered collection of unique elements) instead of a list.

2. **`book.get('author')`**:
   - This part is used to access the value associated with the key `'author'` in each `book` dictionary. The `get` method is a safe way to retrieve values from a dictionary; if the key does not exist, it returns `None` (instead of raising a `KeyError`).
   
3. **`for book in books`**:
   - This loops through each `book` in the iterable `books`. `books` is expected to be a collection of dictionaries, each representing a book with various attributes, including the `'author'`.

4. **`if book.get('author')`**:
   - This condition filters the books. It ensures that only books with a valid (non-`None`) `'author'` value are processed. If `book.get('author')` returns a truthy value (i.e., any non-empty string), the corresponding author is included in the resulting set.

5. **`yield from`**:
   - The `yield from` statement is used within a generator function to yield all values from an iterable. When it encounters the set comprehension on its right side, it will yield each unique author retrieved from the books.

#### Overall Functionality

The overall functionality of this line of code is to create a set of unique authors from a collection of books and yield each author one by one. The use of `set` ensures that each author will only appear once in the output, even if multiple books have the same author.

### Why Use This Code?

1. **Uniqueness**: 
   - By using a set comprehension, the code automatically filters out duplicate authors. If you only used a list comprehension instead, you would have to handle duplicates manually.

2. **Efficiency**:
   - The generator allows for on-demand generation of authors. This can be more memory efficient, especially if `books` is a large collection, because it does not require storing all the authors in memory at once.

3. **Readability**:
   - The use of `yield from` combined with set comprehensions keeps the code concise and expressive. It clearly communicates that the intent is to extract unique authors from a possibly large collection of books in a single line.

### Example in Context

Imagine you have the following list of books:
python
books = [
    {'title': 'Book 1', 'author': 'Author A'},
    {'title': 'Book 2', 'author': 'Author B'},
    {'title': 'Book 3', 'author': 'Author A'},
    {'title': 'Book 4', 'author': None},
    {'title': 'Book 5', 'author': 'Author C'},
]


If you run the line of code you provided in a generator function:
python
def get_unique_authors(books):
    yield from {book.get('author') for book in books if book.get('author')}


And then iterate over the results, you would get the output:
plaintext
Author A
Author B
Author C


### Conclusion

The combination of set comprehension and the `yield from` statement allows for extracting and yielding unique author names from a list of book dictionaries efficiently and in a clean manner. This example illustrates an elegant approach to data extraction while ensuring uniqueness and maintaining readability.

# Ollama Response 

 **Deconstructing a Generative Expression**

The given code snippet is an example of a generative expression in Python, utilizing the `yield from` syntax. Let's break it down to understand what it does.

```python
def get_authors(books):
    return {book.get('author') for book in books if book.get('author')}
```

This function takes a list of dictionaries (`books`) as input and returns a set of authors found in the dictionary. However, the code snippet provided doesn't contain this entire function; instead, it uses `yield from` to generate values on-the-fly.

**The Code Snippet**

```python
yield from {book.get('author') for book in books if book.get('author')}
```

Here's what happens when you run this code:

1. **Generator Expression**: The `{... for ...}` part is a generator expression, which creates an iterator that yields values one at a time.
2. **Set Comprehension**: The `for` clause iterates over the `books` list and applies the condition `if book.get('author')`. This filters out any books without an author.
3. **Dictionary Lookup**: Inside the comprehension, `.get('author')` retrieves the value associated with the key `'author'` for each book in the filtered list. If no such key exists, `.get()` returns `None`.
4. **Yielding Values**: The generator expression uses these values to create a set of authors.

**How `yield from` Works**

The `yield from` statement is used to delegate to a subgenerator or an iterator. When you use `yield from`, the current function yields values one at a time, just like a regular `yield` statement. However, instead of storing all yielded values in memory, it allows the next iteration to start with the last value yielded.

In this case, when we see `yield from {...}`, Python knows that:

* The expression `{...}` is an iterator (the set comprehension).
* We want the function to delegate control to the sub-generator.
* Instead of collecting all values in a list or tuple, we want to yield them one at a time.

**Equivalent Implementation**

To make the code more readable and maintainable, you can rewrite it using `yield` instead of `yield from`:

```python
def get_authors(books):
    for book in books:
        if book.get('author'):
            yield book['author']
```

This implementation yields values one at a time, just like the original code snippet. However, it's easier to understand what's happening when we use a regular `yield` statement instead of `yield from`.

I hope this explanation helped you grasp what the given code does and why! Let me know if you have any further questions or need more clarification. 

# Anthropic Response 

Let's break this code down:

1. This is a generator expression using `yield from` that processes a collection of `books` (likely a list of dictionaries).

2. Let's analyze each part:
   - `book.get('author')`: Safely retrieves the 'author' value from each book dictionary
   - `for book in books`: Iterates through each book in the books collection
   - `if book.get('author')`: Only includes books where the author field exists and is not None
   - The `{...}` creates a set, removing any duplicate authors
   - `yield from` yields each author one at a time from the resulting set

3. Purpose:
   - This code generates a unique list of authors from a collection of books
   - It skips any books that don't have an author field
   - It removes duplicate authors through the set operation
   - It yields authors one at a time (lazy evaluation) rather than creating the whole list in memory

Example usage:
```python
books = [
    {'title': 'Book1', 'author': 'Smith'},
    {'title': 'Book2', 'author': 'Jones'},
    {'title': 'Book3', 'author': 'Smith'},
    {'title': 'Book4'},  # No author
]

# Using the generator
for author in (yield from {book.get('author') for book in books if book.get('author')}):
    print(author)

# Output:
# Smith
# Jones
```

This is a memory-efficient way to process unique authors, especially useful when dealing with large collections of books.

# Cohere Response

 The code you provided is a Python generator expression using the `yield from` syntax. Let's break down the code step by step and explain what it does and why:

```python
yield from {book.get('author') for book in books if book.get('author')}
```

1. `{book.get('author') for book in books if book.get('author')}`— This part of the code is a set comprehension. It iterates over each `book` in the `books` iterable (which is assumed to be a list of dictionaries). For each `book`, it retrieves the value associated with the key `'author'` using the `get` method. The `if book.get('author')` condition checks if the `'author'` key exists in the `book` dictionary and filters out the books that do not have an author. The result of this set comprehension is a set of author values for all the books that contain an author.

   ```python
   {
     'Eric',  # Author of the first book
     'John',  # Author of the second book
     ...
   }
   ```

2. `yield from`— The `yield from` syntax is used to delegate iteration to a sub-generator or an iterable. In this case, it is used to iterate over the set of author values obtained from the set comprehension. When `yield from` encounters an iterable (in this case, the set of authors), it effectively iterates over the elements of that iterable and yields them one by one.

   So, the overall code will yield each author value from the set comprehension, one at a time. This allows the code to generate a series of author values on-the-fly without storing the entire set of authors in memory upfront.

   ```python
   yield 'Eric'
   yield 'John'
   ...
   ```

The purpose of using a generator expression with `yield from` in this context is to efficiently iterate over a large collection of books and extract unique author values without loading the entire collection into memory. This can be particularly useful when dealing with memory constraints or when you want to process the data in a streaming fashion.

In summary, the code retrieves unique author values from a collection of books, filters out books without authors, and yields each author value one by one using a generator expression. This approach provides a memory-efficient way to work with large datasets and enables lazy evaluation, where values are computed only when they are needed.

# Gemini Response 

This code snippet uses a generator expression within a `yield from` statement to efficiently iterate through a list of dictionaries (`books`), extracting and yielding author names. Let's break it down step-by-step:

1. **`books`:** This is assumed to be a list (or iterable) of dictionaries. Each dictionary represents a book and might contain various key-value pairs, including an 'author' key.

2. **`book.get('author')`:** This part accesses the value associated with the 'author' key in each `book` dictionary.  The `.get()` method is crucial here because it handles cases where a book dictionary might *not* have an 'author' key.  If the key is missing, `.get()` returns `None` instead of raising a `KeyError`, preventing the code from crashing.

3. **`if book.get('author')`:** This condition filters the list. It only includes books where `book.get('author')` evaluates to a true value.  In Python, `None` evaluates to `False`, while any string (a non-empty author name) evaluates to `True`. This ensures that only books with authors are processed.

4. **`{book.get('author') for book in books if book.get('author')}`:** This is a *generator expression*. It's a concise way to create a generator that yields author names. It iterates through the `books` list. For each book, if it has an author, the author's name is yielded.  The curly braces `{}`  indicate a set comprehension, which automatically removes duplicates.  If you don't need to remove duplicates, use square brackets `[]` for a list comprehension instead.


5. **`yield from ...`:** This is a statement that delegates the iteration to the generator expression. Instead of explicitly looping through the generator and yielding each value individually, `yield from` efficiently handles the iteration, making the code more readable and potentially more performant.


**In summary:** The code iterates through a list of book dictionaries. For each book that has an author listed, it yields the author's name.  It efficiently handles missing author entries and removes duplicate author names.  The use of a generator expression and `yield from` makes the code compact and avoids creating a large intermediate list in memory, especially beneficial when dealing with a very large `books` list.


**Example:**

Let's say `books` is:

```python
books = [
    {'title': 'Book A', 'author': 'Author X'},
    {'title': 'Book B', 'author': 'Author Y'},
    {'title': 'Book C'},  # Missing author
    {'title': 'Book D', 'author': 'Author X'} # Duplicate author
]
```

Then the code:

```python
author_generator = (yield from {book.get('author') for book in books if book.get('author')})
for author in author_generator:
    print(author)
```

Would output:

```
Author X
Author Y
```

The duplicate "Author X" is removed because of the set comprehension.  If you used a list comprehension `[...]` instead, both "Author X" entries would be present in the output.
